# LSTM network for language modeling
Here we'll define our own forward pass for lstm, and we rely on pytorch for the backward pass.

![LSTM](images/General-scheme-of-an-Long-Short-Term-Memory-neural-networks-LSTM-for-L-p-The.ppm)
<a href=https://www.researchgate.net/figure/General-scheme-of-an-Long-Short-Term-Memory-neural-networks-LSTM-for-L-p-The_fig1_339120709>Image from</a>

In [1]:

import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from torch.utils.data import Dataset
import torch
import numpy as np

class CharDataset(Dataset):
    def __init__(self, data, sequence_len):
        # data in the type of pairs of sentence
        
        chars = ['<pad>']+sorted(list(set(data)))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.ch2i = { ch:i for i,ch in enumerate(chars) }
        self.i2ch = { i:ch for i,ch in enumerate(chars) }
        self.sequence_len = sequence_len
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return len(self.data) - self.sequence_len
    
    def __getitem__(self, idx):
        # grab a chunk of (block_size + 1) characters from the data
        chunk = self.data[idx:idx + self.sequence_len + 1]
        # encode every character to an integer
        dix = [self.ch2i[s] for s in chunk]

        x = torch.tensor(self.vectorization(dix[:-1]), dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        
        
        return x,y
                                                                                                                                       
    def padding(self, string):
        if len(string)<self.sequence_len:
            string =  string + [0]*(self.sequence_len - len(string))
        else:
            string = string[:self.sequence_len]
                   
        return string
    
    def vectorization(self, idx):
        out = np.zeros((self.sequence_len, self.vocab_size))
        for i, index in enumerate(idx):
            out[i,index] = 1
        
        return out

In [3]:
text = open('data/bai_van_mau.txt', 'r', encoding='utf-8').read()
dataset = CharDataset(text, sequence_len=128)

data has 19753491 characters, 236 unique.


# Train network

In [4]:
from model.LSTM import LSTM, ModelConfig
from utils.trainer import Trainer, TrainerConfig

tconfig = TrainerConfig(max_epochs=2, 
                        batch_size=128, 
                        learning_rate=0.01, 
                        grad_norm_clip=1.0, 
                        device='cuda',
                        lr_decay=True, 
                        warmup_tokens=5000, 
                        print_result_n_iter=100, 
                        ckpt_path='model/checkpoint/tfm_translate.net')

mconfig = ModelConfig(vocab_size = dataset.vocab_size,
                        sequence_len = 128,
                        hidden_size = 512,
                        batch_size = tconfig.batch_size,
                        device=tconfig.device)

In [10]:
model = LSTM(mconfig)
trainer = Trainer(model, dataset, tconfig, test_datase t=None, collate=None)

In [ ]:
trainer.train()

epoch: 1 | train loss: 0.11650  | lr: 2.560000e-04:   0%|                       | 1/154323 [00:02<113:58:00,  2.66s/it]

-19. Đến nhà văn đã từng thấy được những kiến thức của mình có thể chung tay để cho chúng ta thấy trong cuộc sống của mình vì người đọc về cuộc sống của con người. Nó có thể là những người trong gia đình có thể tự tin và nhận thức được sự nhân sinh theo người thầy cô đã không có một người bạn thân thiết của mình. Chẳng hạn theo điều kiện thiết thực trong cuộc sống để trở thành một người như vậy, trong thời đại có thể được đó là một trong những câu chuyện thể hiện sự chân thành, tin tưởng và thanh bình, thiếu thốn và tinh thần trong cuộc sống của người thân của tôi là một trong những câu chuyện về những con người có thể thay đổi được tri thức cho chúng ta để làm cho chính mình thì con người ta có thể trở thành một người bạn thân thiết của chúng ta.
Thông thường cho mình một trong những thao tác lập luận của người dân Việt Nam.
Như vậy, trong cách sử dụng là nhữ


epoch: 1 | train loss: 1.19279  | lr: 9.999996e-03:   0%|                      | 101/154323 [00:24<20:47:16,  2.06it/s]

-19 được trang trí trong tiếng nói của mình được trang trí như vậy có thể thấy một chút chứa những chiếc lá có màu nâu sẫm. Cánh hoa thường có màu trắng ngà, trông rất dễ thương của mẹ em. Chú lại ngồi trên ghế nhà trường như vậy nhưng có lẽ những chú chim hồng nhung đỏ rực rỡ và thật đẹp mắt. Bên trong thân cây to và như những cánh hoa đỏ thắm. Những cánh hoa nhỏ xinh xuống biển để làm bằng lăng có màu trắng, những chú chim vàng tròn rất thích thú cho tôi như thế nào? Nhưng thật không có một khu vườn thật là thích thú và nói với em tôi đã thấy người đã thay đổi theo thời gian cho chúng em. Trong bài học đến trường, em cũng có thể trở thành biểu tượng của đất nước có thể thu hút các bạn đang được thưởng thức những thành quả nghiêm trọng nhất là những cái nắng chói chang chảy trôi chảy để đi theo chiếc bút mực của nhà thơ được như một chiếc thuyền nhỏ được trồn


epoch: 1 | train loss: 1.18567  | lr: 9.999973e-03:   0%|                      | 201/154323 [00:46<21:07:42,  2.03it/s]

...

epoch: 1 | train loss: 1.18394  | lr: 9.996782e-03:   1%|▏                    | 1801/154323 [06:48<22:07:05,  1.92it/s]

ÂÂJÂÂÂÂÂJÂJÂJÂJJJJJÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJJÂÂJÂJJJJÂJÂÂÂÂÂÂJJJJJJÂJJÂJJÂJJJÂÂÂJÂÂÂJJÂÂJÂJÂJJÂÂJÂJÂÂJÂJÂÂÂÂJÂJJJJJÂJJJÂJÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂJJJÂÂJÂÂJÂJÂJJÂJJJÂÂJÂÂJÂÂÂÂJÂJÂJÂJÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂJÂJÂÂÂÂJÂJÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂJJÂJJJÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂJÂJÂJÂJÂÂÂÂÂÂÂÂÂÂÂÂÂJÂJJÂJÂJÂJÂÂJJJJJJJÂJJJJÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂJÂÂÂÂJÂJÂÂÂJÂÂÂÂÂJÂJÂJJJJJJJJJÂJJJÂÂJJJJJJJÂJÂJÂJJÂJJJJJJÂJÂJÂJJJJÂJÂJÂJÂJJÂJJÂÂÂJÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂJÂJJJJJJÂJJJÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂJÂÂJÂÂÂJÂÂJÂÂÂÂÂÂÂÂÂÂÂÂJÂJJJJÂJJJÂJJJJJJÂÂJJJÂJJJÂJJJÂJJJJJJJJJÂJJJJJJJÂJÂÂÂÂÂÂÂÂÂJÂJÂJÂJÂJÂÂÂJÂJJÂJÂJÂJÂJJÂÂJJÂÂJÂJÂJÂJÂJÂJÂJÂJJÂJJJÂJÂJÂJÂJJJJJJJÂJJJJJJJJJJJJJJJJJJÂJJÂJÂJJJÂJÂJÂÂÂJJJJÂJJJÂJÂJÂJÂJÂJÂJJJJJJJJJJJJJJJJJJJÂÂÂJÂÂJÂÂÂÂÂÂÂJJJÂÂJÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂÂJÂJÂJÂÂÂÂJÂJÂJJJÂJJÂJÂÂJÂÂÂÂÂÂÂÂÂÂÂJÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂÂJÂJJÂJÂJÂÂÂÂÂÂ


epoch: 1 | train loss: 1.18169  | lr: 9.996407e-03:   1%|▎                    | 1901/154323 [07:11<22:27:24,  1.89it/s]

 thu hút nhiều câu chuyện thì chúng ta có thể kể đến thành phố Hồ Chí Minh còn được thể hiện rõ nhất của chủ nghĩa chính là đạt được tinh thần tự học. Như vậy, chúng ta cần phải cố gắng cho con người ta chỉ biết đi theo thời gian.
Nhà thơ có chút thất bại của mỗi người chúng ta thành công trong công việc. Chính từ đó chỉ là một con người thiên nhiên của người thiên nga đã có thể trở thành con người nhưng đã thay đổi và thiếu thốn tình cảm của con người. Trong lòng người đọc sẽ không thể không ngừng thấm đẫm cho những người con trưởng thông của những người bạn thân thiết.
Một thời gian đã trở thành một trong những bài học cho bản thân chúng ta có thể trở thành một trong những người thân yêu nhất của em. Người cầm quy luật của mình đến lúc như vậy. Nhưng tôi thấy trong trẻo của mẹ em năm nay đã có lần chỉ còn năm cánh chạy vào chân trời. Ngày nào cũng vậy, em đã


epoch: 1 | train loss: 1.18558  | lr: 9.996388e-03:   1%|▎                    | 1906/154323 [07:12<11:13:25,  3.77it/s]

In [87]:
sample = "cháu lên ba"
model.generate_output(sample, dataset, top_k=5, temperature=0.7)

'cháu lên ba để đánh giá thể hiện đầy đủ và đặc biệt của đất nước và cảm xúc về tình thương của con người có ý thức sự chỉ cho những người có những nét chữ đầy đủ nghệ thuật của mình, con người chỉ là nguồn cảm hứng đối với tác phẩm: Như thế hệ trẻ, nhà thơ đã giúp chúng ta có ý thức chiến đấu cho mỗi người đều có thể trở thành con gà mình và người ta còn là người thân cho tôi. Mẹ em có thể trả lời con người chúng ta có thể cảm nhận được những thói hư chiến thắng cho ta thấy được tình cảnh của thơ ca ngợi ca nhiều người có thể làm được những nhận thức tốt đẹp của con người. Từ chính ta ngày một gia đình với nhau để làm việc hành hạ chỉ có thể thấy được cả một con người có thể thấy rất nhiều, được thể hiện được những điều đó có thể là một thứ tri thức của con người, thể hiện được cả một nguồn tốt của mình.\nĐể có thể có thêm nhiều những câu chuyện trong thế giới tri thức củ'

In [74]:
from utils.utils import pickle
#parameters = pickle('checkpoint/lstm', model.parameters()) # save
model.params = pickle('checkpoint/lstm') # load